In [1]:
from dataclasses import dataclass
from typing import List

from alphafed.contractor import (ContractEvent, TaskMessageContractor,
                                 TaskMessageEventFactory)


@dataclass
class QueryStateEvent(ContractEvent):
    """状态查询消息。"""

    TYPE = 'query'

    querier: str  # 查询者 ID

    @classmethod
    def contract_to_event(cls, contract: dict) -> 'QueryStateEvent':
        event_type = contract.get('type')
        querier = contract.get('querier')
        assert event_type == cls.TYPE, f'合约类型错误: {event_type}'
        assert querier and isinstance(querier, str), f'invalid querier: {querier}'
        return QueryStateEvent(querier=querier)


@dataclass
class SyncEvent(ContractEvent):
    """状态同步消息。"""

    TYPE = 'sync'  # 消息类型标识，可以是任意字符串，但要保证在算法流程内部唯一

    aggregator: str  # 参与方 ID
    current_round: int  # 当前训练轮次
    participants: List[str]  # 当前参与者的 ID 列表

    @classmethod
    def contract_to_event(cls, contract: dict) -> 'SyncEvent':
        event_type = contract.get('type')
        aggregator = contract.get('aggregator')
        current_round = contract.get('current_round')
        participants = contract.get('participants')
        assert event_type == cls.TYPE, f'合约类型错误: {event_type}'
        assert aggregator and isinstance(aggregator, str), f'invalid aggregator: {aggregator}'
        assert current_round and isinstance(current_round, int), f'invalid current_round: {current_round}'
        assert participants and isinstance(participants, list), f'invalid participants: {participants}'
        return SyncEvent(aggregator=aggregator, current_round=current_round, participants=participants)


@dataclass
class SyncRespEvent(ContractEvent):
    """状态同步响应。"""

    TYPE = 'sync_resp'

    querier: str  # 查询者 ID

    @classmethod
    def contract_to_event(cls, contract: dict) -> 'SyncRespEvent':
        event_type = contract.get('type')
        querier = contract.get('querier')
        assert event_type == cls.TYPE, f'合约类型错误: {event_type}'
        assert querier and isinstance(querier, str), f'invalid querier: {querier}'
        return SyncRespEvent(querier=querier)


class SyncProcessEventFactory(TaskMessageEventFactory):

    _CLASS_MAP = {  # 在这里添加新消息类型
        QueryStateEvent.TYPE: QueryStateEvent,
        SyncEvent.TYPE: SyncEvent,
        SyncRespEvent.TYPE: SyncRespEvent,
        **TaskMessageEventFactory._CLASS_MAP  # 保留支持 TaskMessageEventFactory 中已有的消息类型
    }


class SyncProcessContractor(TaskMessageContractor):

    def __init__(self, task_id: str):
        super().__init__(task_id=task_id)
        self._event_factory = SyncProcessEventFactory  # 替换为自定义事件工厂

    def query_state(self, querier: str):
        """发送状态查询消息。"""
        event = QueryStateEvent(querier=querier)
        self._new_contract(targets=self.EVERYONE, event=event)  # 指定 self.EVERYONE 广播消息

    def sync_state(self, aggregator: str, current_round: int, participants: List[str], querier: str):
        """发送状态同步消息。"""
        event = SyncEvent(aggregator=aggregator,
                          current_round=current_round,
                          participants=participants)
        self._new_contract(targets=[querier], event=event)  # 指定 querier 消息只会发送给 querier

    def response_sync(self, aggregator: str, querier: str):
        """发送状态同步响应消息。"""
        event = SyncRespEvent(querier=querier)
        self._new_contract(targets=[aggregator], event=event)  # 指定 aggregator 消息只会发送给 aggregator

In [2]:
from alphafed import mock_context

# 参与方模拟脚本

task_id = '79ce0d22-22f8-4f5d-8d0f-35ad0b26db7b'  # 必须与发起方的 task_id 相同
self_node = '2d7514b7-f2fc-4f6c-8d88-849339bd268a'  # 随机指定一个 ID

def sync_state():
    # 实例化合约工具
    contractor = SyncProcessContractor(task_id=task_id)

    print('发送状态查询请求')
    contractor.query_state(querier=self_node)
    print('等待状态同步消息 ...')
    for _event in contractor.contract_events():
        if isinstance(_event, QueryStateEvent) and _event.querier == self_node:
            continue  # 忽略自己刚才群发的消息
        assert isinstance(_event, SyncEvent), '消息类型错误，期望收到状态同步消息'
        aggregator = _event.aggregator
        print('收到状态同步消息')
        print(f'任务发起方为: {aggregator}')
        print(f'当前训练轮次为: {_event.current_round}')
        print(f'当前任务参与方有: {_event.participants}')
        break  # 退出消息监听
    # 发送状态同步响应
    contractor.response_sync(aggregator=aggregator, querier=self_node)
    print(f'与 {aggregator} 同步状态完成.')

# 模拟调试流程
some_others = [
    '8983ba98-74ac-41eb-9588-2c2d57ecf8cb',
    '4c6db456-3ece-4e2f-a165-585d0a6f175c'
]
nodes = [  # 消息广播会发给 nodes 中所有节点，所以需要配置 nodes 才能正常广播消息
    self_node,
    *some_others,
    'd54a3af2-83e2-4da6-bc8d-2dc03634612c' # 发起方ID
]
with mock_context(id=self_node, nodes=nodes):
    sync_state()

发送状态查询请求
等待状态同步消息 ...
收到状态同步消息
任务发起方为: d54a3af2-83e2-4da6-bc8d-2dc03634612c
当前训练轮次为: 11
当前任务参与方有: ['d54a3af2-83e2-4da6-bc8d-2dc03634612c', '8983ba98-74ac-41eb-9588-2c2d57ecf8cb', '4c6db456-3ece-4e2f-a165-585d0a6f175c']
与 d54a3af2-83e2-4da6-bc8d-2dc03634612c 同步状态完成.
